In [832]:
using Printf
using Random
Random.seed!(1234)
m = 3
n = 4
A = rand(1:3, (m,n))
# A = ones(Int64, m,n)
b = rand(1:3,m)
print(A)
println()
print(b)

[1 3 3 3; 2 2 3 2; 1 2 2 3]
[1, 1, 2]

### Idea
---
$$
\begin{gather}
    A' = \begin{bmatrix} A &  b \end{bmatrix} \\
    R' = CA' = \begin{bmatrix} RA &  Cb \end{bmatrix} \\
    R' = \begin{bmatrix} R &  Cb \end{bmatrix} \\
\end{gather}
$$
If $Cb$ is a pivot column, then the system is unsolvable, else it is consistent and has one or more solutions.

In [833]:
function rankconsistency(A::Matrix{Int64}, b::Vector{Int64})
    consistent::Bool = false
    AugmentedMat = Augment(A, b)
    U = UpperEchelon(AugmentedMat)
    R = RowReduction(U)
    return r, U, consistent
end 


rankconsistency (generic function with 1 method)

In [834]:
function Augment(A, b)
    return [A b]
end

Augment (generic function with 1 method)

In [835]:
## Addition Table
function AddGf(i, j)
Add_matrix = [  0 1 2 3  ;  1  0  3  2 ; 2 3 0 1 ; 3 2 1 0 ]
    return Add_matrix[i+1, j+1]
end

AddGf (generic function with 1 method)

In [836]:
## Multiplication table
function MulGf(i, j)
    Mul_matrix = [ 0 0 0 0 ; 0  1  2  3 ; 0  2  3  1 ; 0 3 1 2 ]  
    return Mul_matrix[i+1, j+1]
end

MulGf (generic function with 1 method)

In [837]:
Inv_vec = [1, 3, 2]
function InvGf(i)
    return Inv_vec[i]
end

InvGf (generic function with 1 method)

In [838]:
"""
This function will row reduce an upper Echelon form matrix to a reduced echelon form
    U -> R
"""
function RowReduction(U::Matrix{Int64})
    
    # for i in range()
end

RowReduction

In [839]:
function UpperEchelon(A)
    for i in 1:size(A,1) - 1
        PartialPivot(A, i)
        PivotSubtract(A,i)
    end
end

UpperEchelon (generic function with 1 method)

In [840]:
function PartialPivot(A, row)
    maxIndex = row 
    maxElement = A[row, row]
    isZeroCol = true 
    for i in row:size(A, 1) 
        if(A[i, row] != 0)
            isZeroCol = false
        end

        if(A[maxIndex, row] < A[i, row])
            maxIndex = i
            maxElement = A[i, row]
        else
            continue
        end
    end

    if(row != maxIndex)
    @printf("Exchanging row %d woth row %d", (row, maxIndex))
    temp = A[row,:]
    A[row, :] = A[maxIndex, :]
    A[maxIndex , :] = temp
    end
end

PartialPivot (generic function with 1 method)

In [841]:
#=
* This function takes in the rows i and j assuming row i is a pivot columns and eliminates the entry in row j correspoinding to column i 
* Precondition : Element A(i, i) has an Inverse , i.e partial Pivoting is complete
* Returns: Modified Matrix A after eliminating on all the Rows, below pivot_row.
=#
function PivotSubtract(A, pivot_row)
    for i in pivot_row + 1:size(A, 1)
        RowSubtract(A,pivot_row, i)
    end 
end

PivotSubtract (generic function with 1 method)

In [842]:
#=
* This function takes in the rows i and j assuming row i is a pivot columns and eliminates the entry in row j correspoinding to column i 
* Precondition : Element A(i, i) has an Inverse
* Returns: Modified Matrix A after eliminating on Row j
=#
function RowSubtract(A, i, j)
    @printf("Subtracting Row %d from Row %d \n", i , j)
    factor = MulGf(InvGf(A[i , i]), A[j, i])
    @printf("factor of subtraction = %d \n",factor)
    for index in i:size(A, 2)
        A[j , index] = AddGf(A[j , index], MulGf( A[i, index] ,factor))
    end
end

RowSubtract (generic function with 1 method)

In [843]:
"""
Precondition : Matrix has Non zero element in pivot_row and pivot_col. (depends on the OP of the function PartialPivotNew)
Args : Pivot column and Pivot Row ( Postion of the pivot and then it makes zero , beneath it) 
Returns: Matirx with Reduced Row j.
"""

function RowSubtractNew(A, pivot_row, pivot_col, j)
    @printf("Subtracting Row %d from Row %d \n", pivot_row , j)
    factor = MulGf(InvGf(A[pivot_row , pivot_col]), A[j, pivot_col])
    @printf("factor of subtraction = %d \n",factor)
    for index in pivot_col:size(A, 2)
        A[j , index] = AddGf(A[j , index], MulGf( A[pivot_row, index] ,factor))
    end
end

RowSubtractNew (generic function with 1 method)

In [844]:
"""
This function creates a Pivot by finding the maximum element in the particular pivot column
and then performs an exchange of the row
Precondition: The previous positions are all zero
Args: Matrix and Position of the ***Supposed*** Next Pivot
Returns: Matrix and Boolean. 
Boolean = True if Pivot was found
        = False if No Pivot was found 
"""
function PartialPivotNew(A, sup_pivot_row, sup_pivot_col)
    maxIndex = sup_pivot_row 
    maxElement = A[sup_pivot_row, sup_pivot_col]
    isZeroCol = true 
    for i in sup_pivot_row:size(A, 1) 
        if(A[i, sup_pivot_col] != 0)
            isZeroCol = false
        end

        if(A[maxIndex, sup_pivot_col] < A[i, sup_pivot_col])
            maxIndex = i
            maxElement = A[i, sup_pivot_col]
        else
            continue
        end
    end
    #some optimizations could be done here ?
    if(sup_pivot_row != maxIndex)
    @printf("Exchanging row %d woth row %d", sup_pivot_row, maxIndex)
    temp = A[sup_pivot_row,:]
    A[sup_pivot_row, :] = A[maxIndex, :]
    A[maxIndex , :] = temp
    end

    return A,isZeroCol

end

PartialPivotNew

In [845]:
""" 
Precondition: This function assumes that this row has alread a pivot at pivot_row and pivot_col
Args : The pivot_row
Returns : The Matrix after Finding the Pivot on this Particular row
"""

function PivotSubtractNew(A, pivot_row, pivot_col)
    for i in pivot_row+1:size(A, 1)
        RowSubtractNew(A, pivot_row, pivot_col, i)
    end
end

PivotSubtractNew (generic function with 1 method)

In [846]:
"""
This function creates a row reduced Echelon form of A  using RSNew, PSNew, PPNew.
"""
function UpperEchelonNew(A)
    tag = zeros(Int64, size(A, 2))
    col = 1
    for row in 1:size(A, 1) - 1
        isZeroCol = true
        while(isZeroCol && col <= size(A,2))
            A, isZeroCol = PartialPivotNew(A, row, col)
            if(isZeroCol)
                @printf("The column %d is a Zero Column \n", col)
                col = col + 1
            end
        end

        if(!isZeroCol)
            display(A)
            @printf("(%d, %d) is a pivot position\n", row, col)
            tag[col] = 1
            pivot_row = row
            pivot_col = col
            PivotSubtractNew(A, pivot_row, pivot_col)
            #start next time with the next column, this column has been analyzed3 now
        end

        if(col > size(A, 2))
            #Now no more rows need to be checked since there are no spaces for pivots left( all zeroes beneath me anyway)
            @printf("Ending the Process prematurely at row = %d\n",  row)
            break
        end

        if(row == size(A, 1) - 1)
            #then we need to check the position in which the last row has a pivot(if it has one )
            println("Inside the check for the final Row Pivot")
            for column in col+1:size(A, 2)
                if(A[row+1, column] > 0)
                    tag[column] = tag[column] + 1
                    break #crucial
                end
            end
        end
        
        col = col + 1
    end

    consistent = false
    if(tag[size(A, 2)] == 0)
        consistent = true
    end

    Rank = sum(tag) - tag[size(A, 2)]
    println("Rank = ",Rank)
    println("Consistent = ",consistent)
    println(tag)

    return consistent,Rank,tag

end

UpperEchelonNew

In [847]:
AugmentedMatrix = Augment(A, b)
display(AugmentedMatrix)


3×5 Matrix{Int64}:
 1  3  3  3  1
 2  2  3  2  1
 1  2  2  3  2

In [848]:
B = zeros(Int64, 4, 4)
B[1,1] = 1
B[2,1] = 3
B[2,2] = 2
display(B)
PartialPivotNew(B, 1, 1)
display(B)

4×4 Matrix{Int64}:
 1  0  0  0
 3  2  0  0
 0  0  0  0
 0  0  0  0

Exchanging row 1 woth row 2

4×4 Matrix{Int64}:
 3  2  0  0
 1  0  0  0
 0  0  0  0
 0  0  0  0

In [849]:
UpperEchelonNew(AugmentedMatrix)
display(AugmentedMatrix)

Exchanging row 1 woth row 2

3×5 Matrix{Int64}:
 2  2  3  2  1
 1  3  3  3  1
 1  2  2  3  2

(1, 1) is a pivot position
Subtracting Row 1 from Row 2 
factor of subtraction = 3 
Subtracting Row 1 from Row 3 
factor of subtraction = 3 
Exchanging row 2 woth row 3

3×5 Matrix{Int64}:
 2  2  3  2  1
 0  3  0  2  1
 0  2  1  2  2

(2, 2) is a pivot position
Subtracting Row 2 from Row 3 
factor of subtraction = 3 
Inside the check for the final Row Pivot
Rank = 3
Consistent = true
[1, 1, 1, 0, 0]


3×5 Matrix{Int64}:
 2  2  3  2  1
 0  3  0  2  1
 0  0  1  3  1

In [850]:
# function test_ref(a)
#     a = a + 1
#     println(a)
# end

In [851]:
# b = 3
# test_ref(b)
# print(b)